<a href="https://colab.research.google.com/github/ogyogy/colaboratory/blob/main/annealing_pokemon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 量子アニーリングでポケモンのタイプ相性を最適化

## 概要

ポケモンを1対1で出したときに自分が有利になる組み合わせ最適化問題を量子アニーリングで解く

## 参考

[数理最適化・(量子)アニーリングでFGO攻略 ~ 1 v.s. 1 編 ~ - Qiita](https://qiita.com/github-nakasho/items/0d3cc464a6d9342f13f3)

サーヴァントのクラス相性行列をポケモンのタイプ相性表に置き換える

[バトルに役立つ！ タイプ相性表を公開！｜『ポケットモンスター サン・ムーン』公式サイト](https://www.pokemon.co.jp/ex/sun_moon/fight/161215_01.html)

## 定式化

### バイナリ変数

自分のポケモンを$x$、相手のポケモンを$y$とする

$k$はポケモンのタイプ、$x_k = 1$は自分のポケモンのタイプが$k$であることを表す

$x_{k} = \{0, 1\} \\ y_{k} = \{0, 1\}$

### コスト関数

相手のポケモンのタイプに対して、自分のポケモンのタイプが効果が抜群なら2.0倍、効果が今一つなら0.5倍、それ以外なら1.0倍のダメージとする

QUBOは以下のように表される

$H(q_i) = \sum_{i\geq j} Q_{ij}q_i q_j \\ q_i \in \{0, 1\}$

タイプ相性表を$D_{ij}$とすると、ダメージ倍率の計算式は以下のように表される

$damage = \sum_{i} \left\{ x_i \left( \sum_j D_{ij} y_{j} \right) \right\}$

解くべき問題はダメージ倍率を最大化すること

$\max damage$

最小化問題に変換すると

$\min -damage$

従ってコスト関数は

$cost = -damage$

### 制約条件

簡単のため、お互いのポケモンのタイプは単一であるとする

$\sum_k x_k = 1 \\ \sum_k y_k = 1$

QUBOに合わせて書き換えると

$H_A = \left( \sum_k x_k -1 \right)^2 \\ H_B = \left( \sum_k y_k -1 \right)^2$

2乗するのは制約が破られたときにコストを増加させるため

### QUBO

最終的に解くべきQUBOは以下のように表される

$Q = cost + A H_A + B H_B$

ここで、$A$、$B$はハイパーパラメータ

## プログラム作成

### パッケージのインストール

In [ ]:
!pip install openjij pyqubo

### タイプ相性表の作成

In [2]:
import numpy as np

# タイプの名称
TYPE_NAMES = ['ノーマル', 'ほのお', 'みず', 'でんき', 'くさ', 'こおり',
              'かくとう', 'どく', 'じめん', 'ひこう', 'エスパー', 'むし',
              'いわ', 'ゴースト', 'ドラゴン', 'あく', 'はがね', 'フェアリー']
# ポケモンのタイプの数
N_TYPE = len(TYPE_NAMES)
# タイプ相性表
# 行が自分のタイプ、列が相手のタイプ、値はダメージ倍率
D = np.array([
    # 無   炎   水   電   草   氷   闘   毒   地   飛   超   虫   岩   霊   竜   悪   鋼   精   
    [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 0.0, 1.0, 1.0, 0.5, 1.0], # ノーマル
    [1.0, 0.5, 0.5, 1.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 0.5, 1.0, 0.5, 1.0, 2.0, 1.0], # ほのお
    [1.0, 2.0, 0.5, 1.0, 0.5, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 2.0, 1.0, 0.5, 1.0, 1.0, 1.0], # みず
    [1.0, 1.0, 2.0, 0.5, 0.5, 1.0, 1.0, 1.0, 0.0, 2.0, 1.0, 1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 1.0], # でんき
    [1.0, 0.5, 2.0, 1.0, 0.5, 1.0, 1.0, 0.5, 2.0, 0.5, 1.0, 0.5, 2.0, 1.0, 0.5, 1.0, 0.5, 1.0], # くさ
    [1.0, 0.5, 0.5, 1.0, 2.0, 0.5, 1.0, 1.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 0.5, 1.0], # こおり
    [2.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 0.5, 1.0, 0.5, 0.5, 0.5, 2.0, 0.0, 1.0, 2.0, 2.0, 0.5], # かくとう
    [1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 0.5, 0.5, 1.0, 1.0, 1.0, 0.5, 0.5, 1.0, 1.0, 0.0, 2.0], # どく
    [1.0, 2.0, 1.0, 2.0, 0.5, 1.0, 1.0, 2.0, 1.0, 0.0, 1.0, 0.5, 2.0, 1.0, 1.0, 1.0, 2.0, 1.0], # じめん
    [1.0, 1.0, 1.0, 0.5, 2.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 2.0, 0.5, 1.0, 1.0, 1.0, 0.5, 1.0], # ひこう
    [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 1.0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 0.0, 0.5, 1.0], # エスパー
    [1.0, 0.5, 1.0, 1.0, 2.0, 1.0, 0.5, 0.5, 1.0, 0.5, 2.0, 1.0, 1.0, 0.5, 1.0, 2.0, 0.5, 0.5], # むし
    [1.0, 2.0, 1.0, 1.0, 1.0, 2.0, 0.5, 1.0, 0.5, 2.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 0.5, 1.0], # いわ
    [0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 2.0, 1.0, 0.5, 1.0, 1.0], # ゴースト
    [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 0.5, 0.0], # ドラゴン
    [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 2.0, 1.0, 0.5, 1.0, 0.5], # あく
    [1.0, 0.5, 0.5, 0.5, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 0.5, 2.0], # はがね
    [1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 2.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 0.5, 1.0]  # フェアリー
])

### タイプ相性表の可視化

In [3]:
import pandas as pd

df = pd.DataFrame(D, columns=TYPE_NAMES, index=TYPE_NAMES)
df = df.replace({0.0: '×', 0.5: '△', 1.0: ' ', 2.0: '○'})
df

,ノーマル,ほのお,みず,でんき,くさ,こおり,かくとう,どく,じめん,ひこう,エスパー,むし,いわ,ゴースト,ドラゴン,あく,はがね,フェアリー
ノーマル,,,,,,,,,,,,,△,×,,,△,
ほのお,,△,△,,○,○,,,,,,○,△,,△,,○,
みず,,○,△,,△,,,,○,,,,○,,△,,,
でんき,,,○,△,△,,,,×,○,,,,,△,,,
くさ,,△,○,,△,,,△,○,△,,△,○,,△,,△,
こおり,,△,△,,○,△,,,○,○,,,,,○,,△,
かくとう,○,,,,,○,,△,,△,△,△,○,×,,○,○,△
どく,,,,,○,,,△,△,,,,△,△,,,×,○
じめん,,○,,○,△,,,○,,×,,△,○,,,,○,
ひこう,,,,△,○,,○,,,,,○,△,,,,△,


### QUBOの作成

ベクトルの内積はシグマ記号を用いて以下のように表すことができる

$a \cdot b = \sum_{i} a_i b_i$

ダメージ倍率の計算式は以下のように表される

$damage = \sum_{i} \left\{ x_i \left( \sum_j D_{ij} y_{j} \right) \right\} = x \cdot \left(D \cdot y\right)$


In [4]:
from pyqubo import Array, Constraint

# 自分のポケモンのタイプを表すバイナリ変数
x = Array.create('x', shape=(N_TYPE), vartype='BINARY')
# 相手のポケモンのタイプを表すバイナリ変数
y = Array.create('y', shape=(N_TYPE), vartype='BINARY')
# コスト関数を定義
damage = np.dot(x, np.dot(D, y))
# 制約項を定義
H_A = Constraint((sum(x) - 1) ** 2, label='HA')
H_B = Constraint((sum(y) - 1) ** 2, label='HB')
# ハイパーパラメータを設定
# 小さい値だと制約が破れるので適当な値を設定する必要がある
A = 10
B = 10
# ハミルトニアン全体を定義
Q = -damage + A * H_A + B * H_B
# モデルをコンパイル
model = Q.compile()
qubo, offset = model.to_qubo()

### OpenJijで解く

In [5]:
import openjij as oj

# SQAを用いる
sampler = oj.SQASampler()
# SamplerにQUBOを渡す
response = sampler.sample_qubo(Q=qubo, num_reads=300)
# エネルギーが一番低い状態を取得
dict_solution = response.first.sample
# 得られた結果をデコード
decoded_sample = model.decode_sample(dict_solution, vartype='BINARY')
# 破られている制約が存在する場合表示
broken = decoded_sample.constraints(only_broken=True)
if broken:
    print('制約が破られています: {}'.format(broken))
# 得られた結果をインデックスからタイプの名称に変換して見やすくする
# 正しく実行されると自分から見て効果が抜群の組み合わせが表示される
# 組み合わせは実行ごとに異なる
# 制約が破られているとタイプが表示されなかったり、複数表示されたりする
x_array = []
y_array = []
for i in range(N_TYPE):
    if decoded_sample.array('x', i) == 1:
        x_array.append(TYPE_NAMES[i])
    if decoded_sample.array('y', i) == 1:
        y_array.append(TYPE_NAMES[i])
print('自分のポケモンのタイプ: {}'.format(', '.join(x_array)))
print('相手のポケモンのタイプ: {}'.format(', '.join(y_array)))

自分のポケモンのタイプ: でんき
相手のポケモンのタイプ: みず
